In [1]:
from __future__ import print_function, division

import torch
import torch.nn as nn
from torch.nn import functional as F

import torch.optim as optim
from torch.optim import lr_scheduler

import torchvision
from torchvision import datasets, models, transforms, utils
from torch.utils.data import Dataset, DataLoader

import matplotlib.pyplot as plt
import numpy as np
import cv2
import time
import os
import copy
import random
import time
import pandas as pd
plt.ion()   # interactive mode

In [2]:
seed = 1234
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)

In [3]:
data_dir = 'image/'
test_transforms = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model=torch.load('submission.h5')
model.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [5]:
def predict_image(image):
    image_tensor = test_transforms(image).float()
    image_tensor = image_tensor.unsqueeze_(0)
    input = (image_tensor)
    input = input.to(device)
    output = model(input)
    index = output.data.cpu().numpy().argmax()
    return index

In [6]:
import cv2
import glob

imdir = 'image/'
ext = ['jpeg']    # Add image formats here

files = []
[files.extend(glob.glob(imdir + '*.' + e)) for e in ext]

images_test = [cv2.imread(file) for file in files]

In [7]:
class_names=['dent', 'dent,glass_shatter', 'dent,glass_shatter,head_lamp,scratch', 'dent,glass_shatter,head_lamp,scratch,tail_lamp', 'dent,glass_shatter,scratch', 'dent,head_lamp', 'dent,head_lamp,scratch', 'dent,scratch', 'dent,scratch,tail_lamp', 'dent,tail_lamp', 'glass_shatter', 'head_lamp', 'head_lamp,scratch', 'scratch', 'scratch,tail_lamp', 'tail_lamp', 'unknown']

In [8]:
one_hot_1=[[1,0,0,0,0,0],[1,1,0,0,0,0],[1,1,1,1,0,0],[1,1,1,1,1,0],[1,1,0,1,0,0],[1,0,1,0,0,0],[1,0,1,1,0,0],
           [1,0,0,1,0,0],[1,0,0,1,1,0],[1,0,0,0,1,0],[0,1,0,0,0,0],[0,0,1,0,0,0],[0,0,1,1,0,0],[0,0,0,1,0,0],
           [0,0,0,1,1,0],[0,0,0,0,1,0],[0,0,0,0,0,1]]


In [9]:
len(class_names)

17

In [10]:
predictions_test=[]
for i in range(len(images_test)):
    topil=transforms.ToPILImage()
    predictions_test.append(class_names[predict_image(topil(images_test[i]))])

In [11]:
prediction_tst_df = pd.DataFrame({'image' : files,
                                'Class' : predictions_test}, 
                                columns=['image','Class'])

In [12]:
prediction_tst_df

,image,Class
0,image/1080.jpeg,unknown
1,image/959.jpeg,dent
2,image/1038.jpeg,scratch
3,image/918.jpeg,unknown
4,image/1096.jpeg,unknown
...,...,...
295,image/1040.jpeg,unknown
296,image/976.jpeg,dent
297,image/864.jpeg,"dent,glass_shatter,head_lamp,scratch,tail_lamp"
298,image/921.jpeg,glass_shatter


In [13]:
predictions_test_1hot=[]
for i in range(len(images_test)):
    topil=transforms.ToPILImage()
    predictions_test_1hot.append(one_hot_1[predict_image(topil(images_test[i]))])

In [15]:
df_1 = pd.DataFrame(predictions_test_1hot)

In [16]:
df_1.columns = ['dent', 'glass_Shatter','head_lamp','scratch','tail_lamp','unknown']

In [17]:
df_1

,dent,glass_Shatter,head_lamp,scratch,tail_lamp,unknown
0,0,0,0,0,0,1
1,1,0,0,0,0,0
2,0,0,0,1,0,0
3,0,0,0,0,0,1
4,0,0,0,0,0,1
...,...,...,...,...,...,...
295,0,0,0,0,0,1
296,1,0,0,0,0,0
297,1,1,1,1,1,0
298,0,1,0,0,0,0


In [18]:
fj=prediction_tst_df.join(df_1)

In [19]:
display(fj)

,image,Class,dent,glass_Shatter,head_lamp,scratch,tail_lamp,unknown
0,image/1080.jpeg,unknown,0,0,0,0,0,1
1,image/959.jpeg,dent,1,0,0,0,0,0
2,image/1038.jpeg,scratch,0,0,0,1,0,0
3,image/918.jpeg,unknown,0,0,0,0,0,1
4,image/1096.jpeg,unknown,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...
295,image/1040.jpeg,unknown,0,0,0,0,0,1
296,image/976.jpeg,dent,1,0,0,0,0,0
297,image/864.jpeg,"dent,glass_shatter,head_lamp,scratch,tail_lamp",1,1,1,1,1,0
298,image/921.jpeg,glass_shatter,0,1,0,0,0,0


In [20]:
final_submission=fj.drop(['Class'], axis=1)

In [21]:
final_submission.head(144)

,image,dent,glass_Shatter,head_lamp,scratch,tail_lamp,unknown
0,image/1080.jpeg,0,0,0,0,0,1
1,image/959.jpeg,1,0,0,0,0,0
2,image/1038.jpeg,0,0,0,1,0,0
3,image/918.jpeg,0,0,0,0,0,1
4,image/1096.jpeg,0,0,0,0,0,1
...,...,...,...,...,...,...,...
139,image/991.jpeg,1,0,0,0,0,0
140,image/968.jpeg,1,0,0,0,0,0
141,image/987.jpeg,1,0,0,0,0,0
142,image/895.jpeg,0,0,1,0,0,0


In [ ]:
final_submission.to_excel("prediction.xlsx")

In [22]:
final_submission['sort'] = final_submission['image'].str.extract('(\d+)', expand=False).astype(int)
final_submission.sort_values('sort',inplace=True, ascending=True)
final_submission1 = final_submission.drop('sort', axis=1)
(final_submission1)

,image,dent,glass_Shatter,head_lamp,scratch,tail_lamp,unknown
144,image/801.jpeg,0,0,0,0,1,0
164,image/802.jpeg,0,0,1,0,0,0
221,image/803.jpeg,0,0,0,0,0,1
254,image/804.jpeg,1,0,0,0,0,0
276,image/805.jpeg,1,1,1,1,1,0
...,...,...,...,...,...,...,...
4,image/1096.jpeg,0,0,0,0,0,1
72,image/1097.jpeg,0,1,0,0,0,0
253,image/1098.jpeg,1,0,1,1,0,0
275,image/1099.jpeg,1,0,0,0,0,0


In [23]:
final_final=final_submission1.reset_index()

In [24]:
final_final.drop('index',axis=1,inplace=True)

In [25]:
final_final

,image,dent,glass_Shatter,head_lamp,scratch,tail_lamp,unknown
0,image/801.jpeg,0,0,0,0,1,0
1,image/802.jpeg,0,0,1,0,0,0
2,image/803.jpeg,0,0,0,0,0,1
3,image/804.jpeg,1,0,0,0,0,0
4,image/805.jpeg,1,1,1,1,1,0
...,...,...,...,...,...,...,...
295,image/1096.jpeg,0,0,0,0,0,1
296,image/1097.jpeg,0,1,0,0,0,0
297,image/1098.jpeg,1,0,1,1,0,0
298,image/1099.jpeg,1,0,0,0,0,0


In [25]:
final_final.to_excel("predictionx.xlsx")